# Project 3: Understanding User Behavior

* You're a data scientist at a game development company

* Your latest mobile game has two events you're interested in tracking: buy a sword & join guild

* Each has metadata characterstic of such events (i.e., sword type, guild name, etc)

## Linux Commands Used

#### Navigating into Project 3 folder
cd ~/w205/project-3-jcweaver

#### Copying files
##### One time to copy game_api in
cp ~/w205/course-content/11-Storing-Data-III/game_api.py .

##### After each week of class, run corresponding week copy yml file
cp ~/w205/course-content/11-Storing-Data-III/docker-compose.yml .

cp ~/w205/course-content/12-Querying-Data-II/docker-compose.yml .

cp ~/w205/course-content/13-Understanding-Data/docker-compose.yml .

#### Run each time to start the cluster, create the kafka topic events, and run the game_api file on Flask

docker-compose up -d
docker ps -a

docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

docker-compose exec mids env FLASK_APP=/w205/project-3-jcweaver/game_api.py flask run --host 0.0.0.0

#### Week 12: Use Apache Bench to generate some events for our game app, run each of these 10+ times

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/purchase_a_sword

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/join_guild

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/buy_a_sword

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/join_guild

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/buy_a_sword

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/sell_a_sword_long

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/sell_a_sword_short

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/sell_a_sword_long

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/sell_a_sword_short

#### Week 13: Use Apache Bench to make an infinite loop to run the apache bench command
while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword; sleep 5; done

#### After generating events, run kafkacat to check they made it to kafka
docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e

#### Use Spark to create a symbolic link
docker-compose exec spark bash

ln -s /w205 w205 #create symbolic link

exit

#### Create a jupyter notebook running on a PySpark kernel
docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark


#### Check files made it on hdfs
docker-compose exec cloudera hadoop fs -ls /tmp/sword_purchases

#### Go into hive
docker-compose exec cloudera hive

##### In hive create an external table
create external table if not exists default.sword_purchases (Accept string, Host string, User_Agent string, event_type string, timestamp string) stored as parquet location '/tmp/sword_purchases'  tblproperties ("parquet.compress"="SNAPPY");


#### Go into Presto
docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

##### In Presto
show tables;

select count(*) from sword_purchases;

select * from sword_purchases;

##  PySpark Notebook Setup
Imports,
Spark freebie

In [1]:
import json
from pyspark.sql import Row
from pyspark.sql.functions import udf
import pandas as pd

In [2]:
#spark freebie
spark

## Assembling Data Pipeline
* Use Spark streaming to filter select event types from Kafka
* Land them into HDFS/parquet
* Read them back in to do a simple analysis

### Commands from Week 11
Note the .shows() here for lambdas will dump core and give a stack trace due to differing schemas we learned in weeks 12 and 13.

User defined munge event function that changes the host of events to moe

In [3]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/11-Storing-Data-III/separate_events.py
@udf('string')
def munge_event(event_as_json):
    event = json.loads(event_as_json)
    event['Host'] = "moe"
    event['Cache-Control'] = "no-cache"
    return json.dumps(event)

Raw events read from Kafka

In [4]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/11-Storing-Data-III/separate_events.py
raw_events = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

munged events apply the munge event function on raw_events

In [5]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/11-Storing-Data-III/separate_events.py
munged_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .withColumn('munged', munge_event('raw'))

show what's in the munged events

In [6]:
munged_events.show()

+--------------------+--------------------+--------------------+
|                 raw|           timestamp|              munged|
+--------------------+--------------------+--------------------+
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2021-04-10 01:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|202

extracted_events expands the munged events dataframe to show moe in a host column

In [7]:
extracted_events = munged_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.munged))) \
        .toDF()

show what's in the extracted_events

In [8]:
#Expect core dumps here after Week 11 on shows because of different things on the schema
extracted_events.show()

Py4JJavaError: An error occurred while calling o82.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 6 fields are required while 7 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$.fromJava(EvaluatePython.scala:138)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:235)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:228)
	at org.apache.spark.rdd.RDD$$anonfun$mapPart

In [9]:
#Filtered extracted_events that are purchase_swords
sword_purchases = extracted_events \
.filter(extracted_events.event_type == 'purchase_sword')

In [10]:
#Expect core dumps here on shows if different things on the schema
sword_purchases.show()

Py4JJavaError: An error occurred while calling o127.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 6 fields are required while 7 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$.fromJava(EvaluatePython.scala:138)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apa

In [11]:
#Filtered extracted_events that are defaults
default_hits = extracted_events \
.filter(extracted_events.event_type == 'default')

In [12]:
#Expect core dumps here on shows if different things on the schema
default_hits.show()

Py4JJavaError: An error occurred while calling o172.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 6 fields are required while 7 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$.fromJava(EvaluatePython.scala:138)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apa

### Commands from Week 12 and HW Setup
* From the job filtered_writes.py, pull and modify the code to write the data frames out to parquet format.

* From the pyspark code, pull the code to read a parquet file, register as a temp table, perform some basic sql, and convert to Pandas

In [3]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/12-Querying-Data-II/filtered_writes.py
#Function to check if event is purchase_sword
@udf('boolean')
def is_purchase(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False

In [4]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/12-Querying-Data-II/filtered_writes.py
#Reading raw events from Kafka
raw_events = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [5]:
#Show what's in the raw_events
raw_events.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 48 6F 73 7...|events|        0|     0|2021-04-13 17:02:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     1|2021-04-13 17:02:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     2|2021-04-13 17:02:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     3|2021-04-13 17:02:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     4|2021-04-13 17:02:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     5|2021-04-13 17:02:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     6|2021-04-13 17:02:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     7|2021-04-13 17:02:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0

In [6]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/12-Querying-Data-II/filtered_writes.py
#Use the purchase_swords event filter we defined above
purchase_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .filter(is_purchase('raw'))

In [7]:
#Show what's in the filtered purchase sword events
purchase_events.show()

+--------------------+--------------------+
|                 raw|           timestamp|
+--------------------+--------------------+
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13

In [8]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/12-Querying-Data-II/filtered_writes.py
#Expand purchase_events table by expanding raw into additional columns
extracted_purchase_events = purchase_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw))) \
        .toDF()

In [9]:
#Show what's in the extracted_purchase_events
extracted_purchase_events.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_type|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|us

In [10]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/12-Querying-Data-II/filtered_writes.py
#write the data frames out to parquet format
extracted_purchase_events \
        .write \
        .mode('overwrite') \
        .parquet('/tmp/purchases')

In [11]:
#Copied from https://github.com/kevin-crook-ucb/ucb_w205_crook_supplement/blob/master/2021_Spring/synch_12.md
#Reading in from parquet
purchases = spark.read.parquet('/tmp/purchases')

In [12]:
#Showing what we read in
purchases.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_type|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|us

In [13]:
#Registering as a temp table
purchases.registerTempTable('purchases')

In [14]:
#Running some SQL
purchases_by_example2 = spark.sql("select * from purchases where Host = 'user1.comcast.com'")


In [15]:
#Showing our SQL output
purchases_by_example2.show()


+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_type|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|us

In [16]:
#Converting to pandas
df = purchases_by_example2.toPandas()

In [17]:
#counting the number of hosts
df.groupby("Host").count()

,Accept,User-Agent,event_type,sword_type,timestamp
Host,,,,,
user1.comcast.com,20,20,20,20,20


In [18]:
#User defined boolean function to check if event is join guild
@udf('boolean')
def is_join_guild(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'join_guild':
        return True
    return False

In [19]:
#Filter raw events to join guild events using udf above
join_guild_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .filter(is_join_guild('raw'))

In [20]:
#Expanding out raw into new columns
extracted_join_guild_events = join_guild_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw))) \
        .toDF()

In [21]:
#write the data frames out to parquet format
extracted_join_guild_events \
        .write \
        .mode('overwrite') \
        .parquet('/tmp/join_guilds')

## Some Simple Data Analytics

### Setup
writing data frames out to parquet format, reading parquet format, registering as a temp table, performing some basic SQL, and converting to Pandas

In [22]:
#User defined function to check if sword is in the event type
@udf('boolean')
def is_sword_event(event_as_json):
    event = json.loads(event_as_json)
    if 'sword' in event['event_type']:
        return True
    return False

In [23]:
#Filter using the udf above
sword_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .filter(is_sword_event('raw'))

In [24]:
#Show what's in sword_events
sword_events.show()

+--------------------+--------------------+
|                 raw|           timestamp|
+--------------------+--------------------+
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13 17:02:...|
|{"Host": "user1.c...|2021-04-13

In [25]:
#Expand sword_events
extracted_sword_events = sword_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw))) \
        .toDF()

In [26]:
#Show what's the in extracted_sword_events table
extracted_sword_events.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_type|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|us

In [27]:
#write the data frames out to parquet format
extracted_sword_events \
        .write \
        .mode('overwrite') \
        .parquet('/tmp/swords')

In [28]:
#read a parquet file
swords = spark.read.parquet('/tmp/swords')

In [29]:
#show what we read in from parquet
swords.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_type|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|us

In [30]:
#register as a temp table
swords.registerTempTable('swords')

In [31]:
#Some SQL queries
#How many distinct event types are there for sword actions
long_swords = spark.sql("select count( distinct event_type) from swords")
long_swords.show()

+--------------------------+
|count(DISTINCT event_type)|
+--------------------------+
|                         4|
+--------------------------+



In [32]:
#Some SQL queries
#Only show me sword events where the sword type is long
long_swords = spark.sql("select * from swords where sword_type == 'long'")

In [33]:
#show the result of the sql query above
long_swords.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_type|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|      long|2021-04-13 17:02:...|
|   */*|us

In [34]:
#Convert to Pandas
long_swords_df = long_swords.toPandas()
long_swords_df

,Accept,Host,User-Agent,event_type,sword_type,timestamp
0,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.602
1,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.606
2,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.608
3,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.612
4,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.614
5,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.618
6,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.622
7,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.625
8,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.629
9,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,long,2021-04-13 17:02:20.632


### Some Pandas manipulation and corresponding insights

In [35]:
long_swords_df.groupby("event_type").count()

,Accept,Host,User-Agent,sword_type,timestamp
event_type,,,,,
purchase_sword,40,40,40,40,40
sell_a_sword_long,40,40,40,40,40


There were 40 counts of purchase_sword with long length and 40 counts of sell_a_sword with long length.

In [40]:
long_swords_df.groupby(["Host",'event_type']).agg('count')

Accept  User-Agent  sword_type  timestamp
Host              event_type                                                  
user1.comcast.com purchase_sword         20          20          20         20
                  sell_a_sword_long      20          20          20         20
user2.att.com     purchase_sword         20          20          20         20
                  sell_a_sword_long      20          20          20         20

User1 purchased 20 long swords and sold 20 long swords. User 2 purchased 20 long swords and sold 20 long swords. There are no other users.

In [41]:
#Converting timestamp to a datetime object
long_swords_df.timestamp=pd.to_datetime(long_swords_df.timestamp)

In [42]:
#Get counts of the seconds buckets
long_swords_df.timestamp.dt.second.value_counts()

20    20
40    10
38    10
34    10
32    10
22    10
19    10
Name: timestamp, dtype: int64

If we group events by the second, we can see 7 buckets. Most of these buckets only saw 10 events but the 20 second mark saw 20 events.

## Week 13
* pull and modify the code to write hdfs files in streaming mode
* sink.awaitTermination() needs to be changed to sink.stop() to stop the stream
* markdown cells for hive and presto

In [1]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/13-Understanding-Data/write_swords_stream.py
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType

In [3]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/13-Understanding-Data/write_swords_stream.py

def purchase_sword_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
    ])

In [4]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/13-Understanding-Data/write_swords_stream.py

@udf('boolean')
def is_sword_purchase(event_as_json):
    """udf for filtering events
    """
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False

In [5]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/13-Understanding-Data/write_swords_stream.py

raw_events = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .load()

In [6]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/13-Understanding-Data/write_swords_stream.py

sword_purchases = raw_events \
        .filter(is_sword_purchase(raw_events.value.cast('string'))) \
        .select(raw_events.value.cast('string').alias('raw_event'),
                raw_events.timestamp.cast('string'),
                from_json(raw_events.value.cast('string'),
                          purchase_sword_event_schema()).alias('json')) \
        .select('raw_event', 'timestamp', 'json.*')

In [7]:
#Copied from https://github.com/mids-w205-crook/course-content/blob/master/13-Understanding-Data/write_swords_stream.py

sink = sword_purchases \
        .writeStream \
        .format("parquet") \
        .option("checkpointLocation", "/tmp/checkpoints_for_sword_purchases") \
        .option("path", "/tmp/sword_purchases") \
        .trigger(processingTime="10 seconds") \
        .start()

In [8]:
#To stop sink
sink.stop()

Note see top of notebook for additional linux commands used.